In [1]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
import requests
import datetime, time
from datetime import date

import pandas as pd
import numpy as np
import re
import os, sys

In [2]:
# !pip install chromedriver_autoinstaller

import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

driver = wd.Chrome(driver_name)

In [3]:
# driver.set_window_position(-10000,0) # hide windows
main_url = 'https://kr.investing.com/'
driver.get(main_url)
time.sleep(1)

In [4]:
driver.find_element_by_css_selector('#userAccount > div > a.login.bold').click() 
time.sleep(1)

In [5]:
driver.find_element_by_id('loginForm_password').clear()
driver.find_element_by_id('loginFormUser_email').send_keys('kangeun@naver.com')
time.sleep(1)

In [6]:
driver.find_element_by_id('loginForm_password').clear()
driver.find_element_by_id('loginForm_password').send_keys(os.environ['investing_pass'])
time.sleep(1)

In [7]:
driver.find_element_by_css_selector('#signup > a.newButton.orange').click()  
time.sleep(2)

In [8]:
cpi_url = 'https://kr.investing.com/economic-calendar/cpi-733'

In [9]:
driver.get(cpi_url)

In [10]:
for _ in range(3):  # 최근 데이터 3 pages
    driver.find_element_by_id('showMoreHistory733').click()

In [24]:
df = pd.read_html(driver.page_source)[0] # cpi dataframe

In [25]:
df.columns=  ['annouce_date', 'time', 'real_rate', 'anticipated_rate', 'prev_rate', 'none']

In [26]:
df.head()

,annouce_date,time,real_rate,anticipated_rate,prev_rate,none
0,2022년 07월 13일 (6월),21:30,NaN,8.3%,8.6%,NaN
1,2022년 06월 10일 (5월),21:30,8.6%,8.3%,8.3%,NaN
2,2022년 05월 11일 (4월),21:30,8.3%,8.1%,8.5%,NaN
3,2022년 04월 12일 (3월),21:30,8.5%,8.4%,7.9%,NaN
4,2022년 03월 10일 (2월),22:30,7.9%,7.9%,7.5%,NaN


In [27]:
df['annouce_date'] = df['annouce_date'].apply(lambda x : datetime.datetime.strptime(x[:13], "%Y년 %m월 %d일"))

In [28]:
df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())

In [29]:
df.head()

,annouce_date,time,real_rate,anticipated_rate,prev_rate,none
0,2022-07-13,21:30:00,NaN,8.3%,8.6%,NaN
1,2022-06-10,21:30:00,8.6%,8.3%,8.3%,NaN
2,2022-05-11,21:30:00,8.3%,8.1%,8.5%,NaN
3,2022-04-12,21:30:00,8.5%,8.4%,7.9%,NaN
4,2022-03-10,22:30:00,7.9%,7.9%,7.5%,NaN


In [30]:
today = datetime.date.today()
today = today.strftime('%Y%m%d')

In [33]:
df.to_csv('data/indices_major/aa8_cpi_index_'+today+'.csv', index=False)

In [ ]:
# real_time_future_url = 'https://kr.investing.com/indices/indices-futures' # do not need
# driver.get(real_time_future_url) # do not need